In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
df = pd.read_csv("Bank_Personal_Loan_Modelling.csv")
df.drop(columns=['ID','ZIP Code'], inplace=True)
y = df.pop('Personal Loan')
x = df
x = StandardScaler().fit_transform(x)

In [ ]:
def ant_colony_optimization(x_tr, y_tr, n_ants=10, n_iterations=100, evaporation_rate=0.1, alpha=1, beta=1):
    pheromones = np.ones(x_tr.shape[1])

    for i in range(n_iterations):
        ant_positions = np.zeros((n_ants, x_tr.shape[1]))

        for j in range(x_tr.shape[1]):
            for k in range(n_ants):
                probabilities = (pheromones ** alpha) * ((1 / np.expand_dims(x_tr[:,j], axis=1)) ** beta)
                probabilities *= np.tile(probabilities.sum(axis=1, keepdims=True), (1, x_tr.shape[1]))
                probabilities /= (probabilities.sum(axis=1, keepdims=True) + 1e-8) # Add a small value to avoid division by zero
                cum_probabilities = np.cumsum(probabilities[:,j])
                random_number = np.random.random()
                feature_choice = np.argmax(cum_probabilities >= random_number)
                ant_positions[k, j] = x_tr[feature_choice, j]
                probabilities /= probabilities.sum(axis=1, keepdims=True)

        ant_fitness = np.zeros(n_ants)
        for j in range(n_ants):
            model = Sequential()
            model.add(tf.keras.Input(shape=(x_tr.shape[1],)))
            model.add(Dense(32, activation='relu', weights=[np.repeat(ant_positions[j,:], 32).reshape((11, 32)), np.zeros(32)]))
            model.add(Dense(1,activation='sigmoid', weights=[np.zeros((32, 1)), np.zeros(1)]))
            model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
            _, accuracy = model.evaluate(x_tr, y_tr, verbose=0)
            ant_fitness[j] = accuracy

        pheromones *= (1 - evaporation_rate)
        pheromones += (evaporation_rate * np.mean(ant_positions[ant_fitness == np.max(ant_fitness)], axis=0))

    best_weights = ant_positions[np.argmax(ant_fitness)]
    return best_weights

In [ ]:
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.25)

In [ ]:
best_weights = ant_colony_optimization(x_tr, y_tr)

In [ ]:
model = Sequential()
model.add(tf.keras.Input(shape=(11,)))
model.add(Dense(32,activation='relu', weights=[np.repeat(best_weights, 32).reshape((11, 32)), np.zeros(32)]))
model.add(Dense(1,activation='sigmoid', weights=[np.zeros((32, 1)), np.zeros(1)]))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
model.fit(x=x_tr, y=y_tr, verbose=1, epochs=50)

In [ ]:
# Accuracy without using ACO
accuracy = model.evaluate(x=x_ts, y=y_ts, verbose=0)[1]
print(f"Test accuracy: {accuracy*100:.4f}%")

In [ ]:
# Accuracy using ACO
accuracy = model.evaluate(x=x_ts, y=y_ts, verbose=0)[1]
print(f"Test accuracy: {accuracy*100:.4f}%")